In [1]:
import pandas as pd
import numpy as np
import time
import re
import pickle
import time
import random

In [2]:
tracking2018 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2018.csv")
tracking2019 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2019.csv")
tracking2020 = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/tracking2020.csv")

tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [10]:
play_data = pd.read_csv("/home/amans/Development/scott/nfl-big-data-bowl-2022/csv/plays.csv")
returned_plays = play_data.query("specialTeamsResult == 'Return' and passResult.isnull() and penaltyYards.isnull() and (not returnerId.isnull())", engine='python', inplace=False)


In [19]:
def get_distances(x, player_pos):
    dist = round(np.linalg.norm(player_pos.values - x.values),3)
    return dist

In [20]:
def min_teammate_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index == @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(player_of_interest_row[['x','y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

In [21]:
def min_opponent_distances(player_of_interest_row, time_play_df):
    team = player_of_interest_row['team_index']
    nflid = player_of_interest_row['nflId']
    new_df = time_play_df.query("team_index != @team and displayName != 'football' and nflId != @nflid")
    dists = new_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(player_of_interest_row[['x','y']],))
    min_teammate_dist = min(dists)
    return min_teammate_dist

In [22]:
'''
Function that adds needed features to the play df
'''

def modify_play_df(play_df):
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    if playDirection == 'left':
        play_df.insert(len(play_df.columns),'adj_x', play_df['x'])
        play_df.insert(len(play_df.columns),'adj_y', play_df['y'] - 26.65)
        # change dir and rotation
        play_df.insert(len(play_df.columns), 'adj_o', (play_df['o'] + 270) % 360)
        play_df.insert(len(play_df.columns), 'adj_dir', (play_df['dir'] + 270) % 360)
    if playDirection == 'right':
        
        play_df.insert(len(play_df.columns),'adj_x', 120 - play_df['x'])
        play_df.insert(len(play_df.columns),'adj_y', play_df['y'] - 26.65)
        # change dir and rotation
        play_df.insert(len(play_df.columns), 'adj_o', (play_df['o'] + 90) % 360)
        play_df.insert(len(play_df.columns), 'adj_dir', (play_df['dir'] + 90) % 360)
        
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df.insert(len(play_df.columns), 'sin_adj_o', np.sin(play_df.loc[:,'adj_o']))
    play_df.insert(len(play_df.columns), 'cos_adj_o', np.cos(play_df.loc[:,'adj_o']))
    play_df.insert(len(play_df.columns), 'sin_adj_dir', np.sin(play_df.loc[:,'adj_dir']))
    play_df.insert(len(play_df.columns), 'cos_adj_dir', np.cos(play_df.loc[:,'adj_dir']))
    
    play_df = play_df.fillna(value=0)
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_dis'] = play_df.groupby('displayName').dis.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
   
    return play_df

In [27]:
'''
time_play_df = DF of a specific play at a specific time. 23 rows

playerId = id of player we want to train action for
'''

def store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, playerId):
    # now we can define the values for each row
    rowDict = {}
       
    ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]

    other_team_name = 'home'
    if ball_carrier_team == 'home':
        other_team_name = 'away'

    # get distance from ball carrier
    ball_carrier_state = time_play_df.loc[time_play_df.nflId == returnerId, ['x','y', 's', 'a', 'o', 'dir']]
    time_play_df.insert(len(time_play_df.columns), 'dist_from_ball_carrier', time_play_df.loc[:,['x','y']].apply(get_distances, axis=1,args=(ball_carrier_state.loc[:, ['x','y']],)))

    time_play_df.insert(len(time_play_df.columns), 'ball_carrier_bool', time_play_df.nflId != returnerId)
    time_play_df.insert(len(time_play_df.columns), 'player_bool', time_play_df.nflId != playerId)
    time_play_df.insert(len(time_play_df.columns), 'football_bool', time_play_df.displayName != 'football')
    time_play_df.insert(len(time_play_df.columns), 'team_index', time_play_df.team == ball_carrier_team)

    time_play_df.sort_values(by=["player_bool", "football_bool", "team_index", "dist_from_ball_carrier"], inplace=True, ignore_index=True)

    football_index = time_play_df.index[time_play_df.team == 'football']
    no_football_df = time_play_df.query("displayName != 'football'")
    
    time_play_df.insert(len(time_play_df.columns), 'min_teammate_dist', time_play_df.apply(min_teammate_distances, axis=1, args=(no_football_df,)))
    time_play_df.insert(len(time_play_df.columns), 'min_opponent_dist', time_play_df.apply(min_opponent_distances, axis=1, args=(no_football_df,)))
    
    states = time_play_df.loc[:,['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()
    football_pos = time_play_df.loc[time_play_df.team == 'football',['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']].to_numpy()
    
    rowDict['season'] = season
    rowDict['gameId'] = gameId
    rowDict['playId'] = playId

    rowDict["timeIndex"] = time_index
    rowDict["time"] = time_string
    rowDict["playerId"] = playerId
    rowDict["football_pos"] = football_pos
    # add current state
    rowDict['state'] = states
    # add future states
    next_state = time_play_df.loc[:,['next_x','next_y', 'next_dis', 'next_s', 'next_a', 'next_o', 'next_dir', 'next_adj_x', 'next_adj_y', 'next_adj_o', 'next_adj_dir', 'next_sin_adj_o', 'next_cos_adj_o', 'next_sin_adj_dir', 'next_cos_adj_dir']].to_numpy()
    rowDict['next_state'] = next_state
    # add reward - defined as ball carrier's increase in x
    ball_carrier = time_play_df.query("nflId == @returnerId")
    rowDict['reward'] = round(ball_carrier['adj_x'] - ball_carrier['next_x'],3).values

    action = states[:,:-3]- next_state
    rowDict['action'] = action.round(2)
    
    return rowDict


In [30]:
'''
ball carrier list
'''

start_time = time.time()

#pd.options.mode.chained_assignment = None

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
num_plays = len(returned_plays)

print(f"Num plays = {num_plays}")

for index in range(0, num_plays):


    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']
    returnerId = float(play.loc['returnerId'])

    #####################

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        
    events = list(np.unique(play_df.event))
    
    if 'fumbled' in events:
        print("fumbled: continue")
        continue

    playDirection = play_df.loc[play_df.index[0],'playDirection']
    # modifies in place
    play_df = modify_play_df(play_df)

    # inspect each time

    if 'kick_received' in events or 'punt_received' in events:
        received_time = play_df.query("event == 'kick_received' or event == 'punt_received'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        continue
    if 'tackle' in events or 'out_of_bounds' in events or 'touchdown' in events or 'fumbled' in events:
        tackled_time = play_df.query("event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled'").time.iloc[0]
    else:
        print("Could not get info from play with events ", events)
        continue

    runner_times = play_df.query("time >= @received_time and time <= @tackled_time")

    #####################
    
    # get starting time of play
    #starting_time = min(play_df['time'])
    all_time_steps = runner_times.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):

        try:
            time_play_df = runner_times.query("time == @time_string", inplace=False)
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            rowDict = store_player_info(time_play_df, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, returnerId)
            dict_list.append(rowDict)
                           
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)
    
    if index % 100 == 0:
        print(f"Play {index} out of {num_plays}")
        break

full_train_bc_df = pd.DataFrame(dict_list)

full_train_bc_df.loc[:, 'next_action'] = full_train_bc_df.groupby('playId').action.shift(-1)

full_train_bc_df = full_train_bc_df.dropna(axis=0, how='any', inplace=False)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_bc_df.to_pickle("ball_carrier_data_test.pkl")

print(F"total time: {total_time}  seconds")

Num plays = 4379
Play 0 out of 4379
total time: 9.371  seconds


In [8]:
'''
teammate list
'''

start_time = time.time()

pd.options.mode.chained_assignment = None

# intialize list of dictionaries (State, Action, Reward, New-state)
dict_list = []
num_plays = len(returned_plays)
NUM_TEAMMATES = 8

print(f"Num plays = {num_plays}")

for index in range(0, num_plays):
    RECEIVED = False
    FUMBLED = False
    TACKLED = False

    # get info we want about each play
    play = returned_plays.iloc[index,].copy()
    gameId = play.loc['gameId']
    season = str(gameId)[0:4]
    playId = play.loc['playId']

    #####################
    # returnerId stored as decimal in tracking csv's
    try:
        returnerId = float(play.loc['returnerId'])
    except Exception  as e:
        print(F"Could not get returner id for index {index} at game {gameId} for play {playId}: {Exception}")
        continue
        
    # if no returner id, don't include play
    if pd.isnull(returnerId):
        continue

    # load DF with all plays, to get info about the plays
    if season == '2019':
        play_df = tracking2019.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2018':
        play_df = tracking2018.query('playId == @playId and gameId == @gameId', inplace=False)
    elif season == '2020':
        play_df = tracking2020.query('playId == @playId and gameId == @gameId', inplace=False)
    else:
        print(f"could not find game for game {gameId}, play {playId}")
        continue
        

    #####################
    
    # shift x-axis based on target's endzone
    playDirection = play_df.loc[play_df.index[0],'playDirection']
    
    if playDirection == 'left':
        play_df['adj_x'] = play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 270) % 360
        play_df['adj_dir'] = (play_df['dir'] + 270) % 360
    if playDirection == 'right':
        play_df['adj_x'] = 120 - play_df['x']
        play_df['adj_y'] = play_df['y'] - 26.65
        # change dir and rotation
        play_df['adj_o'] = (play_df['o'] + 90) % 360
        play_df['adj_dir'] = (play_df['dir'] + 90) % 360
        
        
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'o'] = 0
    play_df.loc[play_df.team == 'football', 'dir'] = 0
    play_df.loc[play_df.team == 'football', 'adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'adj_dir'] = 0
    
    play_df['adj_o'].round(2)
    play_df['adj_dir'].round(2)
    
    play_df['sin_adj_o'] = np.sin(play_df['adj_o'])
    play_df['cos_adj_o'] = np.cos(play_df['adj_o'])
    play_df['sin_adj_dir'] = np.sin(play_df['adj_dir'])
    play_df['cos_adj_dir'] = np.cos(play_df['adj_dir'])
    
    # set football's NaN values to zero
    play_df.loc[play_df.team == 'football', 'cos_adj_o'] = 0
    play_df.loc[play_df.team == 'football', 'cos_adj_dir'] = 0
    
    
    play_df.loc[:, 'next_x'] = play_df.groupby('displayName').x.shift(-1)
    play_df.loc[:, 'next_y'] = play_df.groupby('displayName').y.shift(-1)
    play_df.loc[:, 'next_dis'] = play_df.groupby('displayName').dis.shift(-1)
    play_df.loc[:, 'next_s'] = play_df.groupby('displayName').s.shift(-1)
    play_df.loc[:, 'next_a'] = play_df.groupby('displayName').a.shift(-1)
    play_df.loc[:, 'next_o'] = play_df.groupby('displayName').o.shift(-1)
    play_df.loc[:, 'next_dir'] = play_df.groupby('displayName').dir.shift(-1)
    play_df.loc[:, 'next_adj_x'] = play_df.groupby('displayName').adj_x.shift(-1)
    play_df.loc[:, 'next_adj_y'] = play_df.groupby('displayName').adj_y.shift(-1)
    play_df.loc[:, 'next_adj_o'] = play_df.groupby('displayName').adj_o.shift(-1)
    play_df.loc[:, 'next_adj_dir'] = play_df.groupby('displayName').adj_dir.shift(-1)
    play_df.loc[:, 'next_sin_adj_o'] = play_df.groupby('displayName').sin_adj_o.shift(-1)
    play_df.loc[:, 'next_cos_adj_o'] = play_df.groupby('displayName').cos_adj_o.shift(-1)
    play_df.loc[:, 'next_sin_adj_dir'] = play_df.groupby('displayName').sin_adj_dir.shift(-1)
    play_df.loc[:, 'next_cos_adj_dir'] = play_df.groupby('displayName').cos_adj_dir.shift(-1)
    
    # remove NAs created by shifting; i.e. remove plays where there is no next play
    play_df.dropna(axis=0, how='any', subset=['x'], inplace=True)
    
    # create reward - defined as forward progress
    #play_df['reward'] = round(play_df['next_x'] - play_df['x'], 3)
    
    
    #cond = play_df.loc[:,['event', 'displayName', 'position', 'team', 'playDirection', 'next_adj_x', 'next_adj_y', 'next_s', 'next_a', 'next_o', 'next_dir']]
    
    #f = cond.query("displayName == 'football'")
    
    #display(f.iloc[0:50,])
    #events = list(np.unique(f.event))
    #print(f"Events for play {playId} at index {index}")
    #print(events)
    
    # get starting time of play
    starting_time = min(play_df['time'])
    all_time_steps = play_df.loc[:,'time'].unique()

    # for each time step (EACH ROW IN TRAINING SET)
    for time_index, time_string in enumerate(all_time_steps):

        try:
            time_play_df = play_df.query("time == @time_string", inplace=False)
            
            # determine whether we keep the event or not (i.e. if the ball carrier has the ball)
            event = list(np.unique(time_play_df.event))[0]
            if event == 'kick_received' or event == 'punt_received':
                RECEIVED = True
            if event == 'tackle' or event == 'out_of_bounds' or event == 'touchdown' or event == 'fumbled':
                TACKLED = True
            if TACKLED or (not RECEIVED):
                continue
                
            ball_carrier_team = time_play_df.loc[time_play_df.nflId == returnerId, ['team']].values[0][0]
            
            # get list of teammates
            teammate_id_list = time_play_df.query("team == @ball_carrier_team and displayName != 'football' and nflId != @returnerId").nflId
            
            random.seed(2430)
            indices = list(random.sample(range(0, len(teammate_id_list)), NUM_TEAMMATES))
            teammate_id_list = teammate_id_list.iloc[indices]

            # for each teammate, store as one training entry
            for teammate_index, playerId in enumerate(teammate_id_list):
                
                tpd = time_play_df.copy(deep=True)
                rowDict = store_player_info(tpd, time_index, time_string, returnerId, gameId, season, playId, playDirection, ball_carrier_team, playerId)
                dict_list.append(rowDict)
                
                           
        except Exception as e:
            print(f"random error: {e}")
            time.sleep(5)
    
    if index % 250 == 0:
        print(f"Play {index} out of {num_plays}")
    
    if index == 500:
        break

full_train_df = pd.DataFrame(dict_list)

full_train_df.loc[:, 'next_action'] = full_train_df.groupby('playIndex').action.shift(-1)

full_train_df = full_train_df.dropna(axis=0, how='any', inplace=False)

end_time = time.time()
total_time = round(end_time - start_time, 3)

full_train_df.to_pickle("teammate_dataset.pkl")

print(F"total time: {total_time}  seconds")

Num plays = 4494
Play 0 out of 4494
Play 250 out of 4494
Could not get returner id for index 362 at game 2018100701 for play 4281: <class 'Exception'>
Play 500 out of 4494
total time: 20598.391  seconds


In [10]:
len(full_train_df)

172245